### Training Vlads model by first stacking the depths from the Sota models

In [1]:
import sys, os
import torch, wandb
import torch.nn as nn
from torch.utils.data import DataLoader
sys.path.append(os.path.abspath(os.path.join(os.curdir, '..')))
from configs import extended_unet_config as config
from models.unet_convnextv2 import Unet
from datasets.combination_depth_dataset import CombDepthDataset
from utils.train_utils import train_model
import utils.train_utils as tu
from utils.train_utils import evaluate_model
import importlib

/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
# Set a fixed random seed for reproducibility
torch.manual_seed(config.random_seed)

train_full_dataset = CombDepthDataset(
    data_dir=os.path.join(config.dataset_path, 'train/train'),
    depths_dir=config.depth_maps_path,
    list_file=os.path.join(config.dataset_path, 'train_list.txt'),
    transform=config.padded_transform,
    target_transform=config.target_transform,
    has_gt=True,
    depth_model_names=config.depth_model_names,
    uncertainty_dir=None,
    use_uncertainty=None)
    
    # Create test dataset without ground truth
test_dataset = CombDepthDataset(
    data_dir=os.path.join(config.dataset_path, 'test/test'),
    depths_dir=os.path.join(config.depth_maps_path, 'test'),
    list_file=os.path.join(config.dataset_path, 'test_list.txt'),
    transform=config.padded_transform,
    has_gt=False,
    depth_model_names=config.depth_model_names,
    uncertainty_dir=None,
    use_uncertainty=None)  # Test set has no ground truth
    
# Split training dataset into train and validation
total_size = len(train_full_dataset)
train_size = int((1-config.val_part) * total_size)  
val_size = total_size - train_size    
    
train_dataset, val_dataset = torch.utils.data.random_split(
    train_full_dataset, [train_size, val_size]
)
#val_dataset.transform = config.transform_val # I dont think we need to use augmentations for validation


# Create data loaders with memory optimizations
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.train_bs, 
    shuffle=True, 
    num_workers=config.num_workers, 
    pin_memory=True,
    drop_last=True,
    persistent_workers=True,
)
    
    
val_loader = DataLoader(
    val_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)
    
test_loader = DataLoader(
    test_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")

Train size: 20375, Validation size: 3596, Test size: 650


In [3]:
model = config.model()


optimizer = config.optimizer(model.parameters())
print(f"Using device: {config.device}")

Num channels: 7
Using device: cuda:3


In [ ]:
# Wandb logging
exp_name = "extended_unet_sota_depths"

print("Starting training...")
with wandb.init(project="MonocularDepthEstimation",  #CIL
                save_code=True,
                notes=config.WANDB_NOTES):
    model = train_model(model, train_loader, val_loader,
                        config.loss, optimizer, 50, config.device,
                       exp_path=os.path.join(config.dataset_path, exp_name),
                       is_extended_model=True, use_uncertainty_map=False)

Starting training...


wandb: Currently logged in as: noloo to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Epoch 1/50


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [01:28<00:00,  2.53it/s]


Train Loss: 0.3177, Validation Loss: 1.6342
New best model saved at epoch 1 with validation loss: 1.6342
Epoch 2/50


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [01:26<00:00,  2.60it/s]


Train Loss: 0.1964, Validation Loss: 0.2224
New best model saved at epoch 2 with validation loss: 0.2224
Epoch 3/50


Training:   3%|███▉                                                                                                                                                       | 64/2546 [00:33<21:35,  1.92it/s]